# IDM for Unknown Images

In [1]:
# importing modules
import os
import pandas as pd
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

## Functions

In [2]:
# FFT code (adapted from Nederhof)
# compares the FFT results for two images
def dist(fft_real1, fft_imag1, fft_real2, fft_imag2):
    d = 0
    for x in range(size):
        for y in range(size):
            d += abs(fft_real1[x][y] - fft_real2[x][y]) * weight2(x,y)
            d += abs(fft_imag1[x][y] - fft_imag2[x][y]) * weight2(x,y)
    return d

def weight2(x,y):
    if x >= C or y >= C:
        return 0
    if y == 0 and x == 10:
        return g 
    if y == 0 and x == 11:
        return g 
    if y == 0 and x == 12:
        return g 
    if y == 0 and x == 5:
        return g 
    else:
        return D + (C-x) + (C-y)

In [3]:
# IDM code (adapted from Nederhof)

# assume pixels outside of the image are white
def get_safe(im, x, y):
     if x < 0 or x >= size or y < 0 or y >= size:
        return 255
     else:
        return im[x][y]
    
# compares a window of size "context" around a pixel in an 
# image with another window of the same size in another image. Returned is
# the number of pixels where the two windows are distinct.
def distort_window(im1, im2, x1, y1, x2, y2):
    cost = 0
    for xd in range(-context, context+1):
        for yd in range(-context, context+1):
            if get_safe(im1, x1+xd, y1+yd) != get_safe(im2, x2+xd, y2+yd):
                cost += 1
    return cost

# compares windows as above, but warped up to a certain
# distance "warp", taking the optimal cost from all possibilities
def distort(im1, im2, x1, y1):
    best_cost = (1 + 2 * context) * (1 + 2 * context)
    for x2 in range(x1-warp, x1+warp+1):
        for y2 in range(y1-warp, y1+warp+1):
            cost = distort_window(im1, im2, x1, y1, x2, y2)
            if cost < best_cost:
                best_cost = cost
    return best_cost

# compare two images using IDM
def distance(im1, im2):
    diff = 0
    for x1 in range(size):
        for y1 in range(size):
            diff += distort(im1, im2, x1, y1)
    return diff

In [4]:
# actually do the comparison
def IDM(location, bounds, size, C, D, g, fft_thresh, context, warp):
    
    # load new image(s)
    filename = sorted(os.listdir(location))
    if '.DS_Store' in filename:
        filename.remove('.DS_Store')
    new_ratio = np.zeros(len(filename))

    for i in range(len(filename)):
        exec('new' + str(i) + ' = Image.open(location + filename[i])')
        exec("global width,height; width, height = new" + str(i) + ".size")
        new_ratio[i] = height / width
    
    # load in aspect ratio list and pixel values for the data set
    datalist = pd.read_csv('/Users/.../datasetstats.csv', index_col=[0])
    pxls = pd.read_csv('/Users/.../pxls_' + str(size) + '.csv', index_col=[0])
    
    # filter by aspect ratio
    rows, cols = datalist.shape
    
    for j in range(len(filename)):
        exec('saved' + str(j) +  " = []") 
        for i in range(rows):
                if datalist.iat[i,2] < new_ratio[j] + bounds and datalist.iat[i,2] > new_ratio[j] - bounds:
                    name = datalist.iat[i,0]
                    position = name.index('.') # gets position of the _0 in the filename
                    value = name[0:position]
                    exec('saved' + str(j) + '.append(value)')

        exec("print('There are ' + str(len(saved" + str(j) + ")) + ' images not filtered out by aspect ratio.')") # see how many images are left after filtering
        exec('saved_pxls' + str(j) + " = pxls.loc[pxls['0'].isin(saved" + str(j) + ")]") # pixel values for saved images    

    # make the new image the same size as the dataset images and save its pixel values
    for j in range(len(filename)):
        exec("new" + str(j) + "_resize = new" + str(j) + ".resize((size,size))")
        exec("new" + str(j) + "_pxls = np.zeros(size*size)")
        d = 0

        for i in range(size):
            for k in range(size):
                exec("global l,m,n,o; l,m,n,o = new" + str(j) + "_resize.getpixel((k, i))")
                if l > 127.5:
                     exec("new" + str(j) + "_pxls[d] = 255")
                if l < 127.5:
                    exec("new" + str(j) + "_pxls[d] = 0")
                d += 1

    # make the pixels vector a matrix and calculate the new images' FFTs
    for k in range(len(filename)):
        exec("saved_pxls" + str(k) + "= saved_pxls" + str(k) + ".reset_index(drop=True)") # fix axes for saved pixels dataframe
        exec("saved_pxls" + str(k) + "= saved_pxls" + str(k) + ".T.reset_index(drop=True).T")
        exec("new" + str(k) + "_pxls = np.split(new" + str(k) + "_pxls, size)")
        exec("new" + str(k) + "_fft = np.fft.fft(new" + str(k) + "_pxls)")
        
    # compare new image's FFT to saved images' FFTs
    for k in range(len(filename)):
        exec("global new_rows; new_rows = len(saved_pxls" + str(k) + ")")
        exec("FFT_scores" + str(k) + "= np.zeros(new_rows)")
        exec("global saved_pxls; saved_pxls = saved_pxls" + str(k))
        exec("global new_fft; new_fft = new" + str(k) + "_fft")
        for i in range(new_rows):
            temp = saved_pxls.loc[i].to_numpy()
            temp = temp[1:]
            temp = np.split(temp, size)
            fft_temp = np.fft.fft(temp)
            exec("FFT_scores" + str(k) + "[i] = dist(new_fft.real, new_fft.imag, fft_temp.real, fft_temp.imag)")
            
    # sort FFT scores
    for k in range(len(filename)):
        exec("FFT_simil" + str(k) + "= saved_pxls" + str(k) + "[0]")
        exec("FFT_simil" + str(k) + "= pd.DataFrame(FFT_simil" + str(k) +")")
        exec("FFT_scores_df" + str(k) + "= pd.DataFrame(FFT_scores" + str(k) +")")
        exec("FFT_simil" + str(k) + "['FFT Score'] = FFT_scores_df" + str(k))

        exec("FFT_simil" + str(k) + "= FFT_simil" + str(k) + ".sort_values(by=['FFT Score'], ascending=True)") # sort by smallest FFT score first

    # filter out images with FFT_scores larger than the threshold
    for k in range(len(filename)):
        saved = [] 
        exec("global FFT_simil; FFT_simil = FFT_simil" + str(k))
        for i in range(len(FFT_simil)):
                if FFT_simil.iat[i,1] < fft_thresh:
                    value = FFT_simil.iat[i,0]
                    saved.append(value)

        print('There are ' + str(len(saved)) + ' images not filtered out by FFT.') # see how many images are left after filtering
        exec("saved_pxls_FFT" + str(k) + "= saved_pxls" + str(k) + ".loc[saved_pxls" + str(k) + ".loc[:,0].isin(saved)]") # pixel values for saved images
        
    # compare the new image to the dataset
    for k in range(len(filename)):
        exec("saved_pxls_FFT" + str(k) + "= saved_pxls_FFT" + str(k) + ".reset_index(drop=True)") # fix axes for saved pixels dataframe
        exec("saved_pxls_FFT" + str(k) + "= saved_pxls_FFT" + str(k) + ".T.reset_index(drop=True).T")
        exec("global new_rows; new_rows = len(saved_pxls_FFT" + str(k) + ")")
        exec("scores" + str(k) + "= np.zeros(new_rows)")
        exec("global new_pxls; new_pxls = new" + str(k) + "_pxls")
        exec("global saved_pxls; saved_pxls = saved_pxls_FFT" + str(k))

        for j in range(new_rows):
            temp = saved_pxls.loc[j].to_numpy()
            temp = temp[1:]
            temp = np.split(temp, size)
            exec("scores" + str(k) + "[j] = distance(new_pxls, temp)")

        print("Done " + str(k+1) + " out of " + str(len(filename)))
        
    # add similarity scores and sort, then save to result tables
    
    full_results = pd.DataFrame() #initalize result dataframes
    results = pd.DataFrame()

    for i in range(len(filename)):
        exec("simil" + str(i) + " = saved_pxls_FFT" + str(i) + "[0]")
        exec("simil" + str(i) + "= pd.DataFrame(simil" + str(i) +")")
        exec("scores" + str(i) + "_df = pd.DataFrame(scores"+ str(i) +")")
        exec("simil" + str(i) + "['Similarity Score'] = scores" + str(i) + "_df")

        exec("simil" + str(i) + "= simil" + str(i) + ".sort_values(by=['Similarity Score'], ascending=True)") # sort by smallest difference score first
        
        exec("simil" + str(i) + "= simil" + str(i) + ".reset_index(drop=True)") # fix axes for similarity scores dataframe
        exec("simil" + str(i) + "= simil" + str(i) + ".T.reset_index(drop=True).T")
        exec("simil" + str(i) + "= simil" + str(i) + ".rename(columns={0:'Sign', 1:'Similarity Score'})") 
        
        exec("global resulter; resulter = simil" + str(i))
        
        full_results = pd.concat([full_results, resulter], axis=1)
        results = pd.concat([results, resulter.iloc[:,0]], axis=1)
        
    # plot similarity scores
    for i in range(len(filename)):
        exec("plt.plot(simil" + str(i) + ".loc[:,'Similarity Score'], label=str(simil" + str(i) + ".loc[0,'''Sign''']))")
    plt.legend(bbox_to_anchor=(1.04,1), loc="upper left")
    plt.xlabel('Ranking')
    plt.ylabel('Similarity Score')
    plt.title('Similarity Scores of Signs for Input')
    plt.show()
    
    # print the top 10 results and save all the results
    for j in range(len(filename)):
        exec("global top_10; top_10 = simil" + str(j) + ".head(10)")
        place = '/Users/.../' # the whole data set ("Dataset Whole")
        fig = plt.figure(figsize=(20, 7))   

        for i in range(len(top_10)):
            image = Image.open(place + top_10.iat[i,0] + '.png')
            fig.add_subplot(2, 5, i+1)
            plt.axis('off')
            plt.imshow(image)
            plt.title('Number ' + str(i+1)  + '\n' + top_10.iat[i,0] + '\nSimilarity Score = '  + str(top_10.iat[i,1]))
            
    return full_results, results


# Analyze Signs

In [ ]:
# do it with all of one sign
location = '/Users/...' # file with images to be analyzed
bounds = 0.15 # specify range to take above and below aspect ratio
size = 20 # size of the images (20 = best by test)
C = size # FFT constant (size = best by test)
D = 3 # FFT constant (3 = best by test)
g = 1100 #(1100 = best by test)
fft_thresh = 9500000 # threshold for cutting off by FFT values
context = 2 # IDM constant (2 = best by test)
warp = 4 # IDM constant (4 = best by test)

m,n = IDM(location, bounds, size, C, D, g, fft_thresh, context, warp)

# save metrics as .csv files
m.to_csv('/Users/.../signname_fullresults.csv')

n.to_csv('/Users/.../signname_nameresults.csv')